# Libraries

In [75]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [76]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [77]:
%run Data_preprocessing_v1.ipynb

************************************************************
There are 6 set of X
X_train_selected_t1, X_train_selected_t2, X_valid_selected_t1,X_valid_selected_t2,X_test_selected_t1,X_test_selected_t2
------------------------------------------------------------
Normalized version
X_train_selected_t1_norm, X_train_selected_t2_norm, X_valid_selected_t1_norm,X_valid_selected_t2_norm,X_test_selected_t1_norm,X_test_selected_t2_norm
------------------------------------------------------------
There are 4 set of Y
y_train_t1, y_train_t2, y_valid_t1, y_valid_t2
when training, please use: 'y_train_t1_value,y_train_t2_value,y_valid_t1_value,y_valid_t2_value'
************************************************************


In [78]:
print(X_train_selected_t2_norm.shape)
print(X_valid_selected_t2_norm.shape)
print(X_test_selected_t2_norm.shape)

(16760, 54)
(2394, 54)
(4790, 54)


# Hyperparameters

In [79]:
input_size = 54
sequence_length = 1
output_size = 1 # live or dead
num_layers = 3
hidden_size=30
learning_rate = 0.001
batch_size = 64
num_epochs = 500
l2_lamb = 0.0005

# Create JusNN

In [80]:
class JusNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(JusNN, self).__init__()
        
        self.convlayer = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=6, kernel_size=2, stride=1, padding=0),
            nn.AvgPool1d(kernel_size=4,stride=2),
            nn.Conv1d(in_channels=6, out_channels=1, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
        )
        
        self.fclayers = nn.Sequential(
            nn.Linear(input_size, 50),
            nn.ReLU(),
            nn.Linear(50, 24),
        )
        
        self.batchnorm1 = nn.BatchNorm1d(59)
        #self.batchnorm2 = nn.BatchNorm1d(30)
        #self.dropout = nn.Dropout(p=0.5)
        
        self.fc1 = nn.Linear(24, 1)
        
    def forward(self, x):
         x1 = F.leaky_relu(self.fclayers(x))
         #print(x1.shape)
         x2 = self.convlayer(x)
         #print(x2.shape)
            
         x3 = x1+x2
         x3 = self.fc1(x3)
         return x3

# Data transform

In [81]:
class CusDatasetLoader(Dataset):
    def __init__(self,x,y):
        self.len = y.shape[0]
        self.x_data = x
        self.y_data = y
  
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        #return self.dataframe.iloc[index]
        return self.x_data[index], self.y_data[index]

In [82]:
# X_shape(X) : used for split the data into a right tensor size
#input: X in shape of 16760 rows × 7488 columns (df)
#output: X in shape of torch.Size([16760, 312, 24]) (tensor)
def X_shape(X):
    X_tensor = torch.from_numpy(np.array(X)).to(torch.float32)
    X_tensor = torch.split(X_tensor,1)
    X_tensor = torch.stack(X_tensor)
    print("X now in shape of",X_tensor.shape)
    return X_tensor

# y_tensor(y) : used for split the data into a right tensor size
#input: X in shape of 16760 rows × 7488 columns (df)
#output: X in shape of torch.Size([16760, 312, 24]) (tensor)
def y_tensor(y):
    y= torch.from_numpy(np.array(y)).to(torch.float32).reshape(len(y),1)
    print("y now in shape of",y.shape)
    return y

In [83]:
# X should be in smaples, 1, squence, rows
X1 = X_shape(X_train_selected_t2_norm)
y1 = y_tensor(y_train_t2.iloc[:,-1])

train_datasets = CusDatasetLoader(X1, y1)
train_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)

x,y = train_datasets[0]
print(x.shape)
print(y.shape)

X now in shape of torch.Size([16760, 1, 54])
y now in shape of torch.Size([16760, 1])
torch.Size([1, 54])
torch.Size([1])


# Model train

In [84]:
model = JusNN(input_size, hidden_size, num_layers, output_size).to(device)

In [85]:
criterion = nn.MSELoss()
optimizer = optim.Adadelta(model.parameters(), lr = learning_rate)

In [ ]:
losses=[]
acc_list=[]
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.permute(0,1,2)
        #print(data.shape)
        data = data.to(device=device)
        targets = targets.to(device=device)
        targets = torch.squeeze(targets, 1)
        #print(targets)
        
        # forward
        scores = model(data)
        #print(scores.shape)
        loss = criterion(scores, targets)
        l2_lambda = l2_lamb
        l2_norm = sum(p.pow(2.0).sum()
                  for p in model.parameters())
 
        loss = loss + l2_lambda * l2_norm
    
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
    if epoch%5 == 0:
        losses.append(loss.detach().numpy() )
        print(f'Epoch {epoch:03}: | Loss: {loss:.5f} ')
        

Epoch 000: | Loss: 17.83690 
Epoch 005: | Loss: 9.85089 
Epoch 010: | Loss: 6.57690 
Epoch 015: | Loss: 7.96921 
Epoch 020: | Loss: 2.47366 
Epoch 025: | Loss: 5.08515 
Epoch 030: | Loss: 2.88249 
Epoch 035: | Loss: 3.36456 
Epoch 040: | Loss: 2.88411 
Epoch 045: | Loss: 3.46198 
Epoch 050: | Loss: 5.39040 
Epoch 055: | Loss: 3.59967 
Epoch 060: | Loss: 5.10970 
Epoch 065: | Loss: 3.88478 
Epoch 070: | Loss: 4.14644 
Epoch 075: | Loss: 5.04754 
Epoch 080: | Loss: 6.22972 
Epoch 085: | Loss: 4.57679 
Epoch 090: | Loss: 5.56576 
Epoch 095: | Loss: 4.51062 
Epoch 100: | Loss: 2.44994 
Epoch 105: | Loss: 3.18176 
Epoch 110: | Loss: 4.80648 
Epoch 115: | Loss: 4.97720 
Epoch 120: | Loss: 3.21705 
Epoch 125: | Loss: 4.51689 
Epoch 130: | Loss: 4.67668 
Epoch 135: | Loss: 3.42509 
Epoch 140: | Loss: 4.01828 
Epoch 145: | Loss: 4.37903 
Epoch 150: | Loss: 2.94602 
Epoch 155: | Loss: 4.82362 
Epoch 160: | Loss: 2.79944 
Epoch 165: | Loss: 4.17661 
Epoch 170: | Loss: 5.66015 
Epoch 175: | Loss: 

# Model RMSE

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def check_rmse(loader, model):

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for data, targets in loader:
            data = data.permute(0,1,2)
            #print(data.shape)
            data = data.to(device=device)
            targets = targets.to(device=device)
            targets = torch.squeeze(targets, 1)

            scores = model(data)
            res = rmse(scores,targets)
            print(res)
            
    # Toggle model back to train
    model.train()
    
    

In [ ]:
X2 = X_shape(X_valid_selected_t2_norm)
y2 = y_tensor(y_valid_t2.iloc[:,-1])

Valid_datasets = CusDatasetLoader(X2, y2)
Valid_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)
print(f"RMSE on test set: {check_rmse(Valid_loader, model)*100:.2f} %")

# Test

In [ ]:
def test(loader, model):
    # Set model to eval
    model.eval()
    res = torch.tensor([], dtype=torch.int64)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).permute(0,2,1)
            print(x.shape)
            y = torch.squeeze(y.to(device=device), 1)
            print(y.shape)

            scores = model(x)
            _, predictions = scores.max(1)
            res =  torch.cat((predictions, res), 0)  
    model.train()
   
    return res

In [ ]:
X_test

In [ ]:
X_test_final = X_shape(X_test)
y_final = torch.zeros([4790,1])

test_datasets = CusDatasetLoader(X_test_final, y_final)
test_loader = DataLoader(dataset=test_datasets, batch_size=batch_size, shuffle=True)

result = test(test_loader, model)
result

In [ ]:

#df.index = ['Row_1', 'Row_2', 'Row_3', 'Row_4']
res1 = pd.DataFrame(result, columns=["Predicted"])
res1.index = X_test.index
res1.index.name = 'Id'
pd.DataFrame(res1).to_csv('out.csv')


#

#

#